In [1]:
import os
import sys
import pandas as pd
import requests
import re
from datetime import datetime
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
import time
import json
import csv

In [2]:
browser = Chrome()
browser.maximize_window()
base_url = 'https://pann.nate.com'
browser.get(base_url)
browser.find_elements_by_xpath('//*[@id="input_search"]')[0].click()

In [3]:
#원하는 검색어 입력
query_txt = input('크롤링할 키워드를 입력하세요: ')

크롤링할 키워드를 입력하세요: 코로나


In [4]:
browser.find_elements_by_xpath('//*[@id="input_search"]')[0].send_keys(query_txt)

In [5]:
#검색
browser.find_elements_by_xpath('//*[@id="search"]/fieldset/button')[0].click()

In [6]:
#톡톡 게시판 검색 결과 더 보기 선택
browser.find_elements_by_xpath('//*[@id="container"]/div[2]/div[1]/div[3]/p/a')[0].click()
#추후 최신순 선택 추가

In [10]:
nate_page = browser.current_url
page = nate_page + '&page='
#page 주소 부분 keyword 입력 변수 가져오는 형식으로 바꾸기
titles = []
links = []
#page 1-4까지 추출해봄 (50개 게시물이 넘어가면 시간이 다소 걸린다.
#게시글 본문의 'p'와 'espresso_sditor_view'등의 class에 대한 부분 추가해야함.
for i in range(1, 5):
    res = requests.get(page + str(i))
    res.raise_for_status()
    res.encoding = None
    html = res.text
    
    soup = BeautifulSoup(html,'html.parser')
    total_list = soup.find('ul', {'class': 's_list'}).findAll('dt')
    
    for dt in total_list:
        link = dt.find('a')
        real_link = 'https://pann.nate.com' + link.get('href')           
        links.append(real_link)
        titles.append(link.text.strip())
    res = ''
    
print({'제목': titles, 'url': links}, sep='\n')

txt = []

for i in links:
    res = requests.get(i)
    res.raise_for_status()
    res.encoding = None
    html2 = res.text
    
    soup = BeautifulSoup(html2, 'html.parser')
    contentArea = soup.find("div", {"class" : "posting"})
    parags = contentArea.findAll("div", {"id" : "contentArea"})
    content = ""
    for parag in parags:
        content += parag.text.replace('&nbsp;| |\t|\r|\n', '')
    content = content.replace('\n',"")
    content = content.replace('\t',"")
    content = content.replace('\xa0',"")
    content = content.replace('\u200b',"")
    content = content.replace('\ufeff',"")
    txt.append(content)
        
print(txt)

col_name = ['본문']
df = pd.DataFrame(txt, columns = col_name) 
df.to_csv('natepann.csv', index=False, encoding="utf-8-sig")

{'제목': ['여행중에 코로나걸리면 어떡하나요', '이거 코로나일까', '이거 코로나 증상이야?', '코로나 언제끝나', '시험하루전날 코로나 확진', '[진심빡침] 코로나 19 관련 군 기사보고', '...알바가는날인데 코로나검사 받아야돼 문자 이렇게 보내는 거 어때?', '토마토의 나라 스페인에서의 코로나확산 실험-5천 명 밀집', '랖고온 앨범은 진짜 코로나용으로 나온듯', '[속보] 명동서 코로나19 자가격리자 숨진 채 발견', '[속보] 명동서 코로나19 자가격리자 숨진 채 발견', '내 친구 부모님 의료진인데 내 친구 코로나검사 받음', '애들아 코로나인데 시위하려면', '[드루와] 코로나가 점점 가까워지고있는드ㅠ', '[댓글부탁해] 솔직히 코로나 ㅈㄴ 구라같음', '[드루와] 솔직히 나 코로나 상황 이렇게 생각함', '요양병원 코로나검사 횟수 좀 제발 줄여주세요', '[댓글부탁해] 코로나 검사받으면 인정결석이야?알려줘', '[드루와] 코로나 응답하라 1994 버전', '어르신 코로나19 예방접종', '[댓글부탁해] 코로나 증상땜에 결석은 출석인정해줌?', '서울에 코로나좀 더 돌았으면', '난 코로나 때문에 혼자임 친구 없어', '코로나 ㅈ같은 새끼들 진짜', '[신천지예수교회] 비대면 예배로 코로나 19 종식', '[이것좀봐줘] 친구 코로나', '[댓글부탁해] 솔직히 코로나 ㅈㄴ 구라같음', '[댓글부탁해] 코로나 ㅈㄴ 오바하는거같아', '[방탈죄송] 직장에 코로나 확진자 왔다갔는데 우리집에 오겠다는 지인', '[19] 코로나 검사때매 학고 못가면 점수 반영어케됨?', '코로나 일일 확진자 수 구라아님?', '코로나 썩을럼!때문에 미용실을 못감', '갑자기 목 아픈데 코로나 증상??', '배우 손준호 코로나 확진 비상', '[댓글부탁해] 근데 솔직히 코로나 ㅈㄴ 구라같음', '요즘 군대 코로나 대처 수준', '코로나 검사 결과 언제 나오냐 ㅆㅈ', '[댓글부탁해] 우리학교 코로나 확진자 개빡쳐', '시험기간이 코로나를 이기네', '가족이 코로나